# 解析力学と１階化の数式処理

吉田勝俊（宇都宮大学）

### 参考情報
- [Welcome to SymPy’s documentation! &#8212; SymPy 1.10.1 documentation](https://docs.sympy.org/) （本家）
- [SymPy による数式処理とグラフ作成 - 弘前大学 Home Sweet Home](https://home.hirosaki-u.ac.jp/jupyter/sympy/)

In [ ]:
import sympy as sym #数式処理ライブラリ

## ◯ 単振り子の運動方程式

### ■ 数式処理用の文字記号の定義

#### 独立変数とパラメータ

In [ ]:
t, m, l, g = sym.symbols(r't m l g') #引数の文字列はdisplay出力用（TeX表記も使える）
display(t, m, l, g)

#### 時間の関数

In [ ]:
theta = sym.Function(r'\theta')(t) #引数の文字列はdisplay出力用（TeX表記も使える）
display(theta)

### (a) 一般化座標を選ぶ

- 一般化座標: $\theta(t)$

In [ ]:
q = theta

### (b) 座標変換を書き下す

#### 質点の位置ベクトル

In [ ]:
xx = l*sym.Matrix([
    sym.sin(q),
    -sym.cos(q)
]) # この書き方で縦ベクトル

display(xx)

### (c) 全運動エネルギーを書き下す

#### 質点の速度ベクトル

In [ ]:
dxxdt = sym.diff(xx,t)
display(dxxdt)

#### 全運動エネルギー

In [ ]:
T = (m/2)*(dxxdt.dot(dxxdt)) #◯.dot(△)で内積が計算される．
display(T)
T = sym.simplify(T) #数式の整理
display(T)

### (d) 全位置エネルギーを書き下す

In [ ]:
y = xx[1] #位置ベクトルの第1成分（数学の第2成分）
V = m*g*y
display(V)

### (e) $L:=T-V$を公式に代入する

#### 公式（オイラーラグランジュ方程式）：
$\displaystyle
\frac{d}{dt}\left(\frac{\partial L}{\partial \dot \theta}\right)
- \frac{\partial L}{\partial \theta}
= 0
$

In [ ]:
def KouSiki(L, q):
    '''
    オイラー・ラグランジュ方程式（シンプル版）
    - 「左辺=0」の左辺を計算する
    - q は一般化座標（時間の関数）
    '''
    dqdt  = sym.diff(q, t)
    
    LHS = ( #左辺（LHS: Left Hand Side）
        sym.diff( sym.diff(L, dqdt), t)
        - sym.diff(L, q)
    )
    
    return LHS

KouSiki(T - V, theta)

- 単振り子の運動方程式 $ml^2\ddot\theta + mgl\sin\theta = 0$ の左辺が求まった！

## ◯オイラー・ラグランジュ方程式

### 定義：
$\displaystyle
\frac{d}{dt}\left(\frac{\partial L}{\partial \dot q_i}\right)
-\frac{\partial L}{\partial q_i}
+\frac{\partial D}{\partial \dot q_i}
-Q_i
= 0, \quad i=1,2,\cdots
$

### ■ 導出を一括処理するユーザ関数

In [ ]:
def derive_ELE(system):
    '''
    オイラー・ラグランジュ方程式（ELE: Euler-Lagrange equation）
    「左辺＝０」の左辺を導出する関数
    
    def system():    
        q: 一般化座標（ベクトル）
        L: ラグランジュ関数（スカラ）
        D: 散逸関数（スカラ）
        Q: 一般化力（ベクトル）
        
        return (t, q, L, Q, D)
    '''
    t, q, L, D, Q = system()       #一般化座標, ラグランジアン, 散逸関数, 一般化力
    dqdt          = sym.diff(q, t) #一般化座標の時間微分
    dim           = len(q)         #一般化座標の次元
    
    ### ラグランジュの運動方程式 ###
    ELE = sym.Matrix([
        sym.simplify(
            sym.diff( sym.diff(L, dqdt[i]), t )
            - sym.diff(L, q[i])
            + sym.diff(D, dqdt[i])
            - Q[i]
        )
        for i in range(dim)
    ])

    return (ELE, t, q, Q)

## ◯ 台車型倒立振子（CIP: cart inverted pendulum）の運動方程式

### ■システムの定義

- 一般化座標: $(x(t),\theta(t))$

In [ ]:
def system_CIP():
    '''
    システムを定義する関数 （問題に応じて書き換える）
    ※以下は，台車型倒立振子の例
    '''
    # パラメータ: 
    t, M, m, l, g, c1, c2 = sym.symbols(
        't M m l g c_1 c_2', 
        positive=True #正の実数に制限 ※なるべく制限すると simplify がよく効く．以下同．
    )
    G, S = sym.symbols('G S', positive=True)
    
    # 一般化座標（時間関数）: 
    q = sym.Matrix([
        sym.Function(
            r'x', 
            real=True #実数値に制限
        )(t),
        sym.Function(
            r'\theta', 
            real=True #実数値に制限
        )(t),
    ])
    
    # 質点の直交座標
    x, th = q
    xM = sym.Matrix([
        x,
        G
    ])
    xm = sym.Matrix([
        x + l*sym.sin(th),
        l*sym.cos(th) + S,
    ])
    
    dxMdt = sym.diff(xM, t) #その時間微分
    dxmdt = sym.diff(xm, t)
    
    # 運動エネルギー
    T = (M/2)*dxMdt.dot(dxMdt) + (m/2)*dxmdt.dot(dxmdt)
    
    # 位置エネルギー
    h = xm[1] #振子先端の高さ
    U = m*g*h
    
    # ラグランジュ関数
    L = T - U

    # 散逸関数
    # D = (c1/2)*sym.diff(x,t)**2 + (c2/2)*sym.diff(th,t)**2 #粘性減衰
    D = 0 #減衰なし

    # 一般化力
    Q = sym.Matrix([
        sym.Function(r'f', real=True)(t),
        0
    ])
    
    return (t, q, L, D, Q)

### ■運動方程式の導出結果

In [ ]:
EOM, t, q, Q = derive_ELE(system_CIP)

display(EOM) #運動方程式（左辺＝０）の左辺

#### ちなみに，時間変数，一般化座標，一般化力

In [ ]:
display(t, q, Q)

## ◯1階化

In [ ]:
def First_ordered(result):
    '''
    運動方程式を1階化するユーザ関数
    - result := derive_ELE(system)の出力
    '''
    EOMqq, t, q, Q = result

    ### 状態ベクトルの生成 ###
    # 実体
    xq = sym.Matrix(
        [qi for qi in q] 
        + #リストの + は連結
        [sym.diff(qi, t) for qi in q]
    )

    # 通し番号
    xx = sym.Matrix([
        sym.Function(r'x_' + str(i+1), real=True)(t)
        for i in range(len(xq))
    ])

    # 加速度（通し番号）
    aa = sym.Matrix([
        sym.diff(x, t)
        for x in xx
    ])

    ### 状態ベクトルの生成 ###
    dim = len(q)
    EOMxx = []
    for eqn in EOMqq:
        
        #運動方程式を通し番号の状態ベクトルで表す
        for i in range(dim):
            eqn = eqn.subs(xq[dim+i], xx[dim+i])
            eqn = eqn.subs(xq[i], xx[i])
            
        EOMxx.append(eqn)

    ### 加速度について解く ###
    sols = sym.solve(EOMxx, aa[dim:])
    
    ### 1階化した運動方程式 dx/dt = f(x) の右辺 ###
    ffxx = sym.Matrix(
        xx[dim:] 
        + [sols[a].simplify() for a in aa[dim:]]
    )

    return (ffxx, xx, t)

### ■単振り子（SP: simple pendulum）

- 一般化座標: $(\theta(t))$

In [ ]:
def system_SP():
    '''
    システムを定義する関数（単振り子）
    '''
    # パラメータ: 
    t, m, l, g = sym.symbols(
        't m l g', 
        positive=True #正の実数に制限 ※なるべく制限すると simplify がよく効く．以下同．
    )
    c = sym.symbols('c', positive=True)
    
    # 一般化座標（時間関数）: 
    q = sym.Matrix([
        sym.Function(
            r'\theta', 
            real=True #実数値に制限
        )(t),
    ])
    
    # 質点の直交座標
    theta = q[0]
    xx = l*sym.Matrix([
        sym.sin(theta),
        -sym.cos(theta),
    ])
    
    dxxdt = sym.diff(xx, t) #その時間微分
    
    # 運動エネルギー
    T = (m/2)*dxxdt.dot(dxxdt)
    
    # 位置エネルギー
    h = xx[1] #振子先端の高さ
    U = m*g*h
    
    # ラグランジュ関数
    L = T - U

    # 散逸関数
    D = (c/2)*dxxdt.dot(dxxdt) #粘性減衰
#     D = 0 #減衰なし

    # 一般化力
    Q = sym.Matrix([
        0
    ])
    
    return (t, q, L, D, Q)

#### 運動方程式の導出結果

In [ ]:
result_SP = derive_ELE(system_SP)

for _ in result_SP: display(_)

#### 1階化の結果

In [ ]:
first_ordered_SP = First_ordered(result_SP)

display(*first_ordered_SP)

### ■台車型倒立振子（CIP: cart inverted pendulum）

- 一般化座標: $(x(t),\theta(t))$

#### 運動方程式の導出結果

In [ ]:
result_CIP = derive_ELE(system_CIP)

for _ in result_CIP: display(_)

#### 1階化の結果

In [ ]:
first_ordered_CIP = First_ordered(result_CIP)

display(*first_ordered_CIP)